# ADAM Profiling

This notebook profiles the ADAM model to identify speed and memory bottlenecks.

In [1]:
import numpy as np
import cProfile
import pstats
import io
import tracemalloc
from pstats import SortKey

from smooth.adam_general.core.adam import ADAM
from smooth.adam_general.core.es import ES

## Generate Test Data

In [2]:
# Small dataset (quick tests)
np.random.seed(42)
n_small = 120
y_small = 100 + np.cumsum(np.random.randn(n_small)) + 10 * np.sin(np.arange(n_small) * 2 * np.pi / 12)

# Medium dataset
n_medium = 500
y_medium = 100 + np.cumsum(np.random.randn(n_medium)) + 10 * np.sin(np.arange(n_medium) * 2 * np.pi / 12)

# Large dataset
n_large = 2000
y_large = 100 + np.cumsum(np.random.randn(n_large)) + 10 * np.sin(np.arange(n_large) * 2 * np.pi / 12)

print(f"Small: {n_small}, Medium: {n_medium}, Large: {n_large}")

Small: 120, Medium: 500, Large: 2000


## 1. Speed Profiling with cProfile

### 1.1 Profile fit() method

In [3]:
def profile_fit(y, model_spec="AAN", lags=[1, 12], initial="backcasting", n_top=30):
    """Profile the fit method and return stats."""
    profiler = cProfile.Profile()
    
    model = ADAM(model=model_spec, lags=lags, initial=initial)
    
    profiler.enable()
    model.fit(y)
    profiler.disable()
    
    # Create stats
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s).sort_stats(SortKey.CUMULATIVE)
    stats.print_stats(n_top)
    print(s.getvalue())
    
    return model, stats

In [4]:
print("=" * 80)
print("PROFILING FIT - Small dataset, AAN model")
print("=" * 80)
model_small, stats_small = profile_fit(y_small, model_spec="AAN")

PROFILING FIT - Small dataset, AAN model
         9827 function calls (9803 primitive calls) in 0.023 seconds

   Ordered by: cumulative time
   List reduced from 410 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     10/8    0.000    0.000    0.029    0.004 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/estimator.py:280(objective_wrapper)
        2    0.000    0.000    0.012    0.006 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:14(creator)
        2    0.000    0.000    0.012    0.006 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:686(_initialize_states)
        2    0.000    0.000    0.012    0.006 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:770(_initialize_ets_states)
        2    0.000    0.000    0.012    0.006 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/cr

In [5]:
print("=" * 80)
print("PROFILING FIT - Medium dataset, AAN model")
print("=" * 80)
model_medium, stats_medium = profile_fit(y_medium, model_spec="AAN")

PROFILING FIT - Medium dataset, AAN model
         29254 function calls (29216 primitive calls) in 0.079 seconds

   Ordered by: cumulative time
   List reduced from 484 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.004    0.000    0.067    0.007 /usr/lib/python3.13/asyncio/base_events.py:1970(_run_once)
        2    0.006    0.003    0.064    0.032 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:14(creator)
        2    0.000    0.000    0.058    0.029 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:686(_initialize_states)
        4    0.000    0.000    0.041    0.010 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/estimator.py:280(objective_wrapper)
      9/6    0.000    0.000    0.035    0.006 /usr/lib/python3.13/asyncio/events.py:87(_run)
        2    0.000    0.000    0.031    0.016 /home/config/Python/Libraries/sm

In [6]:
print("=" * 80)
print("PROFILING FIT - Model Selection (ZZZ)")
print("=" * 80)
model_select, stats_select = profile_fit(y_small, model_spec="ZZZ")

PROFILING FIT - Model Selection (ZZZ)
         382735 function calls (382353 primitive calls) in 0.854 seconds

   Ordered by: cumulative time
   List reduced from 630 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1628    0.253    0.000    0.421    0.000 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/utils/cost_functions.py:8(CF)
       19    0.000    0.000    0.361    0.019 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:14(creator)
       19    0.000    0.000    0.360    0.019 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:686(_initialize_states)
       19    0.000    0.000    0.360    0.019 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:770(_initialize_ets_states)
       31    0.005    0.000    0.359    0.012 /home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/utils/u

### 1.2 Profile predict() method

In [7]:
def profile_predict(model, h=12, n_top=30):
    """Profile the predict method."""
    profiler = cProfile.Profile()
    
    profiler.enable()
    forecast = model.predict(h=h)
    profiler.disable()
    
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s).sort_stats(SortKey.CUMULATIVE)
    stats.print_stats(n_top)
    print(s.getvalue())
    
    return forecast, stats

In [8]:
print("=" * 80)
print("PROFILING PREDICT")
print("=" * 80)
forecast, pred_stats = profile_predict(model_small, h=12)

PROFILING PREDICT
         3253 function calls (3208 primitive calls) in 0.005 seconds

   Ordered by: cumulative time
   List reduced from 430 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 /usr/share/positron/resources/app/extensions/positron-python/python_files/lib/ipykernel/py3/decorator.py:232(fun)
        1    0.000    0.000    0.002    0.002 /usr/share/positron/resources/app/extensions/positron-python/python_files/lib/ipykernel/py3/IPython/core/history.py:54(only_when_enabled)
        1    0.000    0.000    0.002    0.002 /usr/share/positron/resources/app/extensions/positron-python/python_files/lib/ipykernel/py3/IPython/core/history.py:836(writeout_cache)
        1    0.000    0.000    0.002    0.002 /usr/share/positron/resources/app/extensions/positron-python/python_files/lib/ipykernel/py3/IPython/core/history.py:824(_writeout_input_cache)
        1    0.000    0.000    0.002  

### 1.3 Profile by function category

In [9]:
def profile_fit_detailed(y, model_spec="AAN", lags=[1, 12], initial="backcasting"):
    """Profile fit with detailed breakdown by module."""
    profiler = cProfile.Profile()
    
    model = ADAM(model=model_spec, lags=lags, initial=initial)
    
    profiler.enable()
    model.fit(y)
    profiler.disable()
    
    stats = pstats.Stats(profiler)
    
    # Group by module
    modules = {
        'checker': 0,
        'creator': 0,
        'estimator': 0,
        'forecaster': 0,
        'cost_functions': 0,
        'utils': 0,
        '_adam_general': 0,  # C++ code
        'nlopt': 0,
        'numpy': 0,
        'other': 0
    }
    
    for (filename, line, func), (cc, nc, tt, ct, callers) in stats.stats.items():
        categorized = False
        for module in modules:
            if module in filename:
                modules[module] += ct
                categorized = True
                break
        if not categorized:
            modules['other'] += ct
    
    total = sum(modules.values())
    print("\nTime by module:")
    print("-" * 50)
    for module, time in sorted(modules.items(), key=lambda x: -x[1]):
        pct = 100 * time / total if total > 0 else 0
        print(f"{module:20s}: {time:8.4f}s ({pct:5.1f}%)")
    
    return model, stats

In [10]:
print("=" * 80)
print("TIME BREAKDOWN BY MODULE - Small dataset")
print("=" * 80)
_, _ = profile_fit_detailed(y_small)

TIME BREAKDOWN BY MODULE - Small dataset

Time by module:
--------------------------------------------------
other               :   0.0749s ( 39.8%)
creator             :   0.0525s ( 27.9%)
utils               :   0.0275s ( 14.6%)
estimator           :   0.0160s (  8.5%)
numpy               :   0.0108s (  5.7%)
cost_functions      :   0.0054s (  2.9%)
checker             :   0.0007s (  0.4%)
nlopt               :   0.0001s (  0.0%)
forecaster          :   0.0000s (  0.0%)
_adam_general       :   0.0000s (  0.0%)


In [11]:
print("=" * 80)
print("TIME BREAKDOWN BY MODULE - Medium dataset")
print("=" * 80)
_, _ = profile_fit_detailed(y_medium)

TIME BREAKDOWN BY MODULE - Medium dataset

Time by module:
--------------------------------------------------
other               :   0.4814s ( 75.5%)
creator             :   0.0655s ( 10.3%)
numpy               :   0.0401s (  6.3%)
utils               :   0.0332s (  5.2%)
cost_functions      :   0.0114s (  1.8%)
estimator           :   0.0039s (  0.6%)
checker             :   0.0020s (  0.3%)
nlopt               :   0.0001s (  0.0%)
forecaster          :   0.0000s (  0.0%)
_adam_general       :   0.0000s (  0.0%)


## 2. Memory Profiling with tracemalloc

In [12]:
def profile_memory(y, model_spec="AAN", lags=[1, 12], initial="backcasting", n_top=20):
    """Profile memory usage during fit."""
    tracemalloc.start()
    
    model = ADAM(model=model_spec, lags=lags, initial=initial)
    model.fit(y)
    
    snapshot = tracemalloc.take_snapshot()
    tracemalloc.stop()
    
    # Top memory consumers by line
    print(f"\nTop {n_top} memory allocations by line:")
    print("-" * 80)
    top_stats = snapshot.statistics('lineno')
    for stat in top_stats[:n_top]:
        print(stat)
    
    # Top memory consumers by file
    print(f"\nTop {n_top} memory allocations by file:")
    print("-" * 80)
    top_stats = snapshot.statistics('filename')
    for stat in top_stats[:n_top]:
        print(stat)
    
    return model, snapshot

In [13]:
print("=" * 80)
print("MEMORY PROFILING - Small dataset")
print("=" * 80)
_, snapshot_small = profile_memory(y_small)

MEMORY PROFILING - Small dataset

Top 20 memory allocations by line:
--------------------------------------------------------------------------------
/home/config/Python/Libraries/smooth/python/.venv/lib/python3.13/site-packages/numpy/lib/_arraysetops_impl.py:358: size=4192 B, count=8, average=524 B
/home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/estimator.py:356: size=2200 B, count=7, average=314 B
/home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:3006: size=2032 B, count=2, average=1016 B
/home/config/Python/Libraries/smooth/python/.venv/lib/python3.13/site-packages/numpy/_core/numeric.py:385: size=2032 B, count=2, average=1016 B
/home/config/Python/Libraries/smooth/python/.venv/lib/python3.13/site-packages/numpy/_core/numeric.py:232: size=2016 B, count=2, average=1008 B
/home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/adam.py:465: size=1520 B, count=1, average=1520 B
/home/config/Python/Libraries/smoo

In [14]:
print("=" * 80)
print("MEMORY PROFILING - Large dataset")
print("=" * 80)
_, snapshot_large = profile_memory(y_large)

MEMORY PROFILING - Large dataset

Top 20 memory allocations by line:
--------------------------------------------------------------------------------
/home/config/Python/Libraries/smooth/python/.venv/lib/python3.13/site-packages/numpy/lib/_arraysetops_impl.py:358: size=62.8 KiB, count=8, average=8044 B
/home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/creator.py:3006: size=31.4 KiB, count=2, average=15.7 KiB
/home/config/Python/Libraries/smooth/python/.venv/lib/python3.13/site-packages/numpy/_core/numeric.py:385: size=31.4 KiB, count=2, average=15.7 KiB
/home/config/Python/Libraries/smooth/python/.venv/lib/python3.13/site-packages/numpy/_core/numeric.py:232: size=31.3 KiB, count=2, average=15.7 KiB
/home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/checker.py:1732: size=15.7 KiB, count=2, average=8048 B
/home/config/Python/Libraries/smooth/python/src/smooth/adam_general/core/checker.py:1656: size=15.7 KiB, count=2, average=8048 B
/home/confi

## 3. Timing Individual Components

In [23]:
import time

def time_components(y, model_spec="AAN", lags=[1, 12], initial="backcasting"):
    """Time individual components of the fitting process."""
    from smooth.adam_general.core.checker import parameters_checker
    from smooth.adam_general.core.creator import architector, creator, initialiser
    from smooth.adam_general.core.estimator import estimator
    
    timings = {}
    
    # Create model instance
    model = ADAM(model=model_spec, lags=lags, initial=initial)
    
    # Time parameters_checker
    start = time.perf_counter()
    checked = parameters_checker(
        data=y,
        model=model.model,
        lags=model.lags,
        constant=model.constant,
        outliers=model.outliers,
        occurrence=model.occurrence,
        distribution=model.distribution,
        loss=model.loss,
        h=model.h,
        holdout=model.holdout,
        persistence=model.persistence,
        phi=model.phi,
        initial=model.initial,
        arma=model.arma,
        ic=model.ic,
        bounds=model.bounds,
        regressors=model.regressors,
        X=None
    )
    timings['parameters_checker'] = time.perf_counter() - start
    
    # Time architector
    start = time.perf_counter()
    arch = architector(
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        observations_dict=checked['observations_dict'],
        persistence_checked=checked['persistence_checked'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        explanatory_checked=checked['explanatory_dict'],
        phi_dict=checked['phi_dict'],
        constants_checked=checked['constant_dict'],
        profiles_recent_table=None
    )
    timings['architector'] = time.perf_counter() - start
    
    # Time creator
    start = time.perf_counter()
    created = creator(
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        profiles_dict=arch['profiles_dict'],
        observations_dict=checked['observations_dict'],
        components_dict=arch['components_dict'],
        persistence_checked=checked['persistence_checked'],
        phi_dict=checked['phi_dict'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        constants_checked=checked['constant_dict'],
        explanatory_checked=checked['explanatory_dict']
    )
    timings['creator'] = time.perf_counter() - start
    
    # Time initialiser
    start = time.perf_counter()
    init = initialiser(
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        profiles_dict=arch['profiles_dict'],
        components_dict=arch['components_dict'],
        observations_dict=checked['observations_dict'],
        persistence_checked=checked['persistence_checked'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        explanatory_checked=checked['explanatory_dict'],
        phi_dict=checked['phi_dict'],
        constants_checked=checked['constant_dict'],
        general=checked['general'],
        adam_created=created
    )
    timings['initialiser'] = time.perf_counter() - start
    
    # Time estimator
    start = time.perf_counter()
    est = estimator(
        y=checked['observations_dict']['y_in_sample'],
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        profiles_dict=arch['profiles_dict'],
        components_dict=arch['components_dict'],
        observations_dict=checked['observations_dict'],
        persistence_checked=checked['persistence_checked'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        explanatory_checked=checked['explanatory_dict'],
        phi_dict=checked['phi_dict'],
        constants_checked=checked['constant_dict'],
        general=checked['general'],
        adam_created=created,
        B=init['B'],
        Bl=init['Bl'],
        Bu=init['Bu']
    )
    timings['estimator'] = time.perf_counter() - start
    
    # Print results
    total = sum(timings.values())
    print("\nComponent timings:")
    print("-" * 50)
    for name, t in sorted(timings.items(), key=lambda x: -x[1]):
        pct = 100 * t / total if total > 0 else 0
        print(f"{name:20s}: {t:8.4f}s ({pct:5.1f}%)")
    print("-" * 50)
    print(f"{'TOTAL':20s}: {total:8.4f}s")
    
    return timings

In [22]:
print("=" * 80)
print("COMPONENT TIMINGS - Small dataset")
print("=" * 80)
timings_small = time_components(y_small)

COMPONENT TIMINGS - Small dataset


TypeError: parameters_checker() got an unexpected keyword argument 'y'

In [24]:
print("=" * 80)
print("COMPONENT TIMINGS - Medium dataset")
print("=" * 80)
timings_medium = time_components(y_medium)

COMPONENT TIMINGS - Medium dataset


TypeError: parameters_checker() got an unexpected keyword argument 'arma'

In [ ]:
print("=" * 80)
print("COMPONENT TIMINGS - Large dataset")
print("=" * 80)
timings_large = time_components(y_large)

## 4. Optimization Iterations Profiling

In [ ]:
def profile_cost_function(y, model_spec="AAN", lags=[1, 12], initial="backcasting", n_calls=100):
    """Profile the cost function that's called during optimization."""
    from smooth.adam_general.core.checker import parameters_checker
    from smooth.adam_general.core.creator import architector, creator, initialiser, filler
    from smooth.adam_general.core.utils.cost_functions import CF
    
    # Setup model
    model = ADAM(model=model_spec, lags=lags, initial=initial)
    
    checked = parameters_checker(
        y=y, model=model.model, lags=model.lags, orders=model.orders,
        constant=model.constant, formula=model.formula, outliers=model.outliers,
        level=model.level, occurrence=model.occurrence, distribution=model.distribution,
        loss=model.loss, h=model.h, holdout=model.holdout, persistence=model.persistence,
        phi=model.phi, initial=model.initial, arma=model.arma, ic=model.ic,
        bounds=model.bounds, regressors=model.regressors, X=None
    )
    
    arch = architector(
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        observations_dict=checked['observations_dict'],
        persistence_checked=checked['persistence_checked'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        explanatory_checked=checked['explanatory_dict'],
        phi_dict=checked['phi_dict'],
        constants_checked=checked['constant_dict'],
        profiles_recent_table=None
    )
    
    created = creator(
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        profiles_dict=arch['profiles_dict'],
        observations_dict=checked['observations_dict'],
        components_dict=arch['components_dict'],
        persistence_checked=checked['persistence_checked'],
        phi_dict=checked['phi_dict'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        constants_checked=checked['constant_dict'],
        explanatory_checked=checked['explanatory_dict']
    )
    
    init = initialiser(
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        profiles_dict=arch['profiles_dict'],
        components_dict=arch['components_dict'],
        observations_dict=checked['observations_dict'],
        persistence_checked=checked['persistence_checked'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        explanatory_checked=checked['explanatory_dict'],
        phi_dict=checked['phi_dict'],
        constants_checked=checked['constant_dict'],
        general=checked['general'],
        adam_created=created
    )
    
    B = init['B']
    
    # Time single CF call
    start = time.perf_counter()
    for _ in range(n_calls):
        CF(
            B=B,
            model_type_dict=checked['model_type_dict'],
            lags_dict=checked['lags_dict'],
            profiles_dict=arch['profiles_dict'],
            components_dict=arch['components_dict'],
            observations_dict=checked['observations_dict'],
            persistence_checked=checked['persistence_checked'],
            initials_checked=checked['initials_checked'],
            arima_checked=checked['arima_checked'],
            explanatory_checked=checked['explanatory_dict'],
            phi_dict=checked['phi_dict'],
            constants_checked=checked['constant_dict'],
            general=checked['general'],
            adam_created=created
        )
    elapsed = time.perf_counter() - start
    
    print(f"\nCost function timing ({n_calls} calls):")
    print(f"  Total time: {elapsed:.4f}s")
    print(f"  Per call:   {elapsed/n_calls*1000:.4f}ms")
    print(f"  Calls/sec:  {n_calls/elapsed:.1f}")
    
    # Profile single call
    print(f"\nDetailed profile of single CF call:")
    profiler = cProfile.Profile()
    profiler.enable()
    CF(
        B=B,
        model_type_dict=checked['model_type_dict'],
        lags_dict=checked['lags_dict'],
        profiles_dict=arch['profiles_dict'],
        components_dict=arch['components_dict'],
        observations_dict=checked['observations_dict'],
        persistence_checked=checked['persistence_checked'],
        initials_checked=checked['initials_checked'],
        arima_checked=checked['arima_checked'],
        explanatory_checked=checked['explanatory_dict'],
        phi_dict=checked['phi_dict'],
        constants_checked=checked['constant_dict'],
        general=checked['general'],
        adam_created=created
    )
    profiler.disable()
    
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s).sort_stats(SortKey.CUMULATIVE)
    stats.print_stats(20)
    print(s.getvalue())
    
    return elapsed / n_calls

In [ ]:
print("=" * 80)
print("COST FUNCTION PROFILING - Small dataset")
print("=" * 80)
cf_time_small = profile_cost_function(y_small, n_calls=100)

In [ ]:
print("=" * 80)
print("COST FUNCTION PROFILING - Large dataset")
print("=" * 80)
cf_time_large = profile_cost_function(y_large, n_calls=100)

## 5. Comparison: Different Model Configurations

In [ ]:
def benchmark_configurations(y, configs):
    """Benchmark different model configurations."""
    results = []
    
    for name, config in configs.items():
        model = ADAM(**config)
        
        # Time fit
        start = time.perf_counter()
        model.fit(y)
        fit_time = time.perf_counter() - start
        
        # Time predict
        start = time.perf_counter()
        model.predict(h=12)
        predict_time = time.perf_counter() - start
        
        results.append({
            'name': name,
            'fit_time': fit_time,
            'predict_time': predict_time,
            'total_time': fit_time + predict_time
        })
        
        print(f"{name:30s}: fit={fit_time:.3f}s, predict={predict_time:.3f}s")
    
    return results

In [ ]:
configs = {
    'ANN (simple)': {'model': 'ANN', 'lags': [1], 'initial': 'backcasting'},
    'AAN (trend)': {'model': 'AAN', 'lags': [1], 'initial': 'backcasting'},
    'ANA (seasonal)': {'model': 'ANA', 'lags': [1, 12], 'initial': 'backcasting'},
    'AAA (full)': {'model': 'AAA', 'lags': [1, 12], 'initial': 'backcasting'},
    'MAM (multiplicative)': {'model': 'MAM', 'lags': [1, 12], 'initial': 'backcasting'},
    'ZZZ (selection)': {'model': 'ZZZ', 'lags': [1, 12], 'initial': 'backcasting'},
}

print("=" * 80)
print("BENCHMARK CONFIGURATIONS - Small dataset")
print("=" * 80)
results = benchmark_configurations(y_small, configs)

## 6. Summary and Recommendations

In [ ]:
print("=" * 80)
print("PROFILING SUMMARY")
print("=" * 80)
print("""
Key findings from profiling:

1. SPEED BOTTLENECKS:
   - Check the 'estimator' timing - this includes optimization
   - The cost function (CF) is called many times during optimization
   - filler() is called inside CF and can be slow if not optimized
   - C++ adam_fitter should be fast; if Python overhead is high, look at data conversion

2. MEMORY BOTTLENECKS:
   - Large matrices (mat_vt, mat_wt, mat_f) can consume memory
   - Check for unnecessary copies of arrays
   - Model selection (ZZZ) fits multiple models, multiplying memory usage

3. OPTIMIZATION TARGETS:
   - Reduce dictionary lookups in hot paths
   - Pre-allocate arrays instead of creating new ones
   - Consider caching repeated calculations
   - Ensure numpy operations are vectorized

Run the cells above to identify specific bottlenecks in your environment.
""")